# Break Tests

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.signal as sig
import scipy.stats as stats
from fractions import Fraction
import matplotlib.pyplot as plt

In [2]:
sns.set_theme()

## Understanding Break Tests

Break tests, in Memoirs of the Old World, happen when any unit takes a wound.
2D6 is rolled and compared to the unit's leadership (Ld)

If it's at or below Ld, the unit stays firm and can even counter attack

If it exceeds it, the unit breaks, and makes a forced retreat for every point it breaks by up to its max flee range.
However, for every hex retreated the unit can nullify one wound.

Two factors influence a break test score.

* Add 1 to the dice roll for every wound that the unit could take
* Subtract 1 from the dice roll for every allied unit adjacent to this unit

Additionally, rolling an unmodified double 1, “insane courage”, always holds the unit while an unmodified double 6 always breaks.

In [34]:
D6 = stats.randint(1, 7).pmf(range(1, 7))
TWOD6 = sig.convolve(D6, D6)
BREAK_TABLE = (pd.DataFrame(zip(range(2, 13), TWOD6),
                            columns=["roll", "prob"])
               .set_index("roll"))
BREAK_TABLE

,prob
roll,
2,0.027778
3,0.055556
4,0.083333
5,0.111111
6,0.138889
7,0.166667
8,0.138889
9,0.111111
10,0.083333


In [39]:
def calc_break_roll(ld, wounds=0, allies=0):
    roll = ld - wounds + allies
    if ld < 2:
        roll = 2
    if ld > 12:
        roll = 12
    return roll+1

In [123]:
def break_states(ld, wounds=0, flee=0, allies=0):
    break_roll = calc_break_roll(ld, wounds, allies)
    pass_break = [("pass", roll, 0, wounds, BREAK_TABLE.loc[roll]["prob"])
                  for roll in range(2, break_roll)]
    flee_break = [("flee", break_roll+f-1, f, wounds-f, BREAK_TABLE.loc[break_roll+f-1]["prob"]) 
                  for f in range(1, min(wounds, flee)+1)]
    over_break = [("over", roll, flee, max(wounds-flee, 0), BREAK_TABLE.loc[roll]["prob"])
                  for roll in range(break_roll+min(wounds, flee), 13)]
    return pd.DataFrame(pass_break+flee_break+over_break,
                        columns=["state", "roll", "flee", "wounds", "prob"])

In [145]:
def break_dist(ld, wounds=0, flee=0, allies=0):
    return (break_states(ld, wounds, flee, allies)
            .groupby(["flee", "wounds", "state"])["prob"].sum()
            .sort_index())

In [148]:
break_dist(7, 3, 2)

flee  wounds  state
0     3       pass     0.166667
1     2       flee     0.111111
2     1       flee     0.138889
              over     0.583333
Name: prob, dtype: float64